In [88]:
def copy_to_redshift(cursor,item,keyword,date,table,dwh_iam_role):
    qry = f'''
      TRUNCATE {table};
      copy {table} from 's3://santal/{item}/{keyword}/{date}.json'
      credentials '{dwh_iam_role}'
      format as json 'auto'
    '''
    
    cursor.execute(qry)

In [108]:
def staging_all_tables(keyword,date,redshift):
    conn_string = "postgresql://{}:{}@{}:{}/{}".format(redshift['DWH_DB_USER'], redshift['DWH_DB_PASSWORD'], redshift['DWH_ENDPOINT'], redshift['DWH_PORT'],redshift['DWH_DB'])
    con = psycopg2.connect(conn_string)
    cur = con.cursor()
    
    copy_to_redshift(cur,'tweets',keyword,date,'staging_tweets',redshift['DWH_IAM_ROLE'])
    copy_to_redshift(cur,'users',keyword,date,'staging_users',redshift['DWH_IAM_ROLE'])
    copy_to_redshift(cur,'places',keyword,date,'staging_places',redshift['DWH_IAM_ROLE'])
    
    print("finished staging to redshift")
    